In [ ]:
!pip install autoPyTorch[forecasting]

In [ ]:
from autoPyTorch.api.time_series_forecasting import TimeSeriesForecastingTask
import pandas as pd

In [ ]:
data = pd.read_csv('covid_deaths_clean.csv').set_index('date')

targets = data[['T266']]
features = data[[col for col in data.columns if col != 'T266']]

In [ ]:
forecasting_horizon = 5

y_train = [targets[: -forecasting_horizon]]
y_test = [targets[-forecasting_horizon:]]

X_train = [features[: -forecasting_horizon]]
known_future_features = list(features.columns)
X_test = [features[-forecasting_horizon:]]

In [ ]:
api = TimeSeriesForecastingTask(seed=0, n_jobs=5, 
    #output_directory='model', delete_output_folder_after_terminate=False, 
    ensemble_size=10, ensemble_nbest=10)

api.search(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test, 
    optimize_metric='mean_MAPE_forecasting',
    n_prediction_steps=forecasting_horizon,
    #memory_limit=16 * 1024,  # Currently, forecasting models use much more memories
    func_eval_time_limit_secs=500,
    total_walltime_limit=600,
    #min_num_test_instances=1000,  # proxy validation sets. This only works for the tasks with more than 1000 series
    known_future_features=known_future_features,
)

In [ ]:
test_sets = api.dataset.generate_test_seqs()
y_pred = api.predict(test_sets)
print(y_test, y_pred)